# Design of Experiments for the bioassay (6p)
This exercise is an example of Design of Experiments (DOE). In DOE our aim is to design an experiment such that we obtain as much information as possible from the measurement(s). This is extremely important if the individual experiments are slow and/or expensive. For example, it is economically unsound to stop the production processes of a factory for a long time. It is also possible to perform DOE stepwise by computing after each experiment what is the next most useful experiment.

Use the bioassay experiment data and model. The interesting quantity is the value of LD50. Consider a situation, where the current posterior uncertainty of LD50 is too high for public authorities. In order to avoid exposing animals to poison unnecessarily, compute which dose level in a new experiment would reduce the posterior variance in LD50 most.

For simplicity, the new experiment will involve only a single rat which will be exposed to one of the dose levels specified in the book. Your task is to decide which dose level should be used in the experiment.

Compute with each dose level how the posterior would change if the animal 
1) died
2) survived.
Compute with each dose level what is the probability that the animal
1) dies
2) survives.

Combine the information into the expected utility (reduction in the posterior variance).

Hints and further information:

* For integrating over the posterior distribution of the parameters, use the adaptive Simpson quadrature method. This is illustrated in the Matlab template file bioassay_doe.m, which uses the following helper functions: bioassayp.m and dblquadvec.m. Python users, see bioassay_doe.py.
* Quadrature methods are suitable for fairly smooth 1-3 dimensional integrals and they are often faster than grid sampling or MCMC. In this particular problem, for example, the Monte Carlo error in the MCMC-integration may be so large that we would need a fairly large number of samples in order to get high enough precision. With a quadrature algorithm, however, the integration is fast and the amount of code you have to write for implementation is less than 10 lines. This way you can concentrate on the actual evaluations of the experiment design problem.

A couple of comments on the Design of Experiments

Design of experiments can be also done sequentially one experiment at a time if the cost of an experiment is high. The next experiment is designed after the result of the first experiment is known. This way the maximum amount of information is gained. The experiments are continued until the desired accuracy is reached.

Alternatively it is possible to do multiple experiments at a time if the cost of the experiments is not significantly higher this way. In an animal experiment one can think that each animal has high cost, but doing the experiments sequentially would take more time. If the results are needed quickly it is reasonable to do multiple experiments at a time.

In [32]:
import numpy as np
import scipy.integrate as integ
from scipy.special import expit
from scipy.stats import binom

# this is a template file for the DOE-exercise for Bayesian data analysis.
def bioassayp(a,b,x,y,n):
    '''unnormalized posterior density for bioassay'''
    
    
    t = a+b*x
    et = np.exp(t)
    z = et/(1.+et)

    # Ensure that log(z) and log(1-z) are computable
    eps = 1e-16
    z = np.minimum(z, 1.-eps)
    z = np.maximum(z, eps)
    
    p = np.exp(np.sum(y*np.log(z)+ (n-y)*np.log(1.-z)))
    return p
    

# Bioassay data, (BDA3 page 74)
x = np.array([-.86, -.30, -.05, .73])
n = np.array([5., 5., 5., 5.])
y = np.array([0., 1., 3., 5.])

# Define function that computes the normalization term by integrating function
# bioassayp.m with the given data x,y,n over the region a=[-4,8], b=[1,40] 
# (outside this region, density should be very small) 
Z = lambda x,y,n: integ.dblquad( lambda b,a: bioassayp(a,b,x,y,n), -4., 8., lambda a: 1., lambda a: 40.)[0]

# Create a helper function for evaluating expectation of 
# a function g(a,b) over the posterior of a and b given x, y and n 
# note that normalization term Z is taken into account
bioint = lambda g,x,y,n: integ.dblquad(lambda b,a: g(a,b)*bioassayp(a,b,x,y,n), -4., 8., lambda a: 1., lambda a: 40.)[0] / Z(x,y,n)

# Using bioint function defined above, you can compute the
# expectation of any function g(a,b). For example, compute the
# posterior expectation of LD50, where LD50=g(-a/b)
# bioint integrates over the posterior of a and b given x, y and n
# using lambda function 'lambda a,b: -a/b' which computes -a/b
# (You can compare the result with results from the grid-sampling,
#  normal approximation and Monte Carlo approximation.)
ld50_mean_old = bioint(lambda a,b: -a/b, x, y, n)
ld50_var_old = bioint(lambda a,b: (-a/b - ld50_mean_old)**2, x, y, n)

# When computing other expectations, replace '-a/b' with
# any equation involving a,b,x,y, or n. That equation implements
# function of a and b, where x,y, and n are available constants.

# Note that new experiments can easily be added to previous experiments
# with concatenation. For example, computing expected LD50 given
# result of the new experiment with a dose xt and no death as a result
nn = np.hstack((n, 1))

for dose in x:
    xx = np.hstack((x, dose))
    
    prob = np.zeros(2)
    utility = np.zeros(2)
    
    for outcome in [0, 1]:
        prob[outcome] = bioint(lambda a,b: binom.pmf(outcome, 1, expit(a + b * dose)), x, y, n)
        
        yy = np.hstack((y, outcome))
        ld50_mean = bioint(lambda a,b: -a/b, xx, yy, nn)
        ld50_var = bioint(lambda a,b: (-a/b - ld50_mean)**2, xx, yy, nn)
        
        utility[outcome] = ld50_var_old - ld50_var

    print "dose: %f\t expected utility: %f" % (dose, np.sum(prob*utility))


dose: -0.860000	 expected utility: 0.000012
dose: -0.300000	 expected utility: 0.000344
dose: -0.050000	 expected utility: 0.000901
dose: 0.730000	 expected utility: 0.000172
